In [2]:
import re
import math

In [4]:
RANGE_PATTERN = re.compile(r'(\d+)\.+(\d+)')
XY_COORD_PATTERN_STR = r'([xy])=([\d\.]+)'
COORD_PATTERN_STR = r'{0}, {0}'.format(XY_COORD_PATTERN_STR)
COORD_PATTERN = re.compile(COORD_PATTERN_STR)

In [5]:
COORD_PATTERN.findall('x=452, y=1077..1087')

[('x', '452', 'y', '1077..1087')]

In [11]:
def read_range(coord):
    matches = RANGE_PATTERN.findall(coord)
    if not matches:
        return range(int(coord), int(coord)+1)
    match = matches[0]
    return range(int(match[0]), int(match[1])+1)

def read_coordinates_from_file(filepath):
    coordinates = []
    with open(filepath) as file:
        for i, line in enumerate(file.readlines()):
            matches = COORD_PATTERN.findall(line)
            if not matches:
                raise ValueError('Couldnt parse line #{}: {}'.format(i, line))
            match = matches[0]
            if match[0] == 'x':
                x, y = match[1], match[3]
            else:
                x, y = match[3], match[1]
            x_range, y_range = read_range(x), read_range(y)
            coordinates.append((x_range, y_range))
    return coordinates

In [32]:
def read_grid_from_file(filepath):
    coords = read_coordinates_from_file(filepath)
    x_min, x_max = math.inf, -math.inf
    y_min, y_max = math.inf, -math.inf
    # get size of grid
    for x_range, y_range in coords:
        for x in x_range:
            for y in y_range:
                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
    # create grid
    width = (x_max-x_min+1)+2 # add one buffer to each side
    x_min -= 1 # move min over one to account for padding
    height = y_max-y_min+1
    grid = [['.' for _ in range(width)] for _ in range(height)]
    for x_range, y_range in coords:
        for x in x_range:
            for y in y_range:
                grid[y-y_min][x-x_min] = '#'
    return (grid, (x_min, y_min))

In [79]:
def read_raw_grid_from_file(filepath):
    grid = []
    with open(filepath) as file:
        for line in file.readlines():
            line = line.replace('\n', '')
            grid.append(line)
    return (grid, (0, 0))

In [28]:
def write_grid_to_file(filepath, grid):
    with open(filepath, 'w') as file:
        for row in grid:
            file.write(''.join(row)+'\n')

In [55]:
def grid_to_string(grid):
    return '\n'.join(''.join(row) for row in grid)

In [160]:
def fill_grid(grid, offset, source):
    grid = [[i for i in row] for row in grid] # copy grid
    width, height = len(grid[0]), len(grid)
    x_off, y_off = offset
    x_src, y_src = source[0]-x_off, source[1]-y_off
    # move source down
    if y_src < 0:
        y_src = 0
    # start filling
    grid[y_src][x_src] = '+'
    def recursive_fill(x, y):
        # if out of bounds, ignore
        if y >= height:
            return
        # at current source, keep filling until spilling
        y_initial = y
        while y < height-1 and y >= y_initial:
            cell = grid[y+1][x]
            # if falling through empty air
            if cell == '.':
                grid[y+1][x] = '|'
                y += 1
            # if hitting a base or pool of water
            # fill the current level
            elif cell == '#' or cell == '~':
                grid[y][x] = '~'
                left_spill, right_spill = None, None
                left_wall, right_wall = None, None
                left_x, right_x = x, x
                # search left for spill or wall
                while True:
                    # check if spilling
                    below_cell = grid[y+1][left_x]
                    if below_cell == '.':
                        left_spill = left_x
                        break
                    # check for walls
                    left_cell = grid[y][left_x-1]
                    if left_cell == '#':
                        left_wall = left_x-1
                        break
                    left_x -= 1
                # search right for spill or wall
                while True:
                    # check if spilling
                    below_cell = grid[y+1][right_x]
                    if below_cell == '.':
                        right_spill = right_x
                        break
                    # check for walls
                    right_cell = grid[y][right_x+1]
                    if right_cell == '#':
                        right_wall = right_x+1
                        break
                    right_x += 1
                # if enclosed, fill
                if left_wall is not None and right_wall is not None:
                    for wall_x in range(left_wall+1, right_wall):
                        grid[y][wall_x] = '~'
                    y -= 1
                # if spilt on either side
                else:
                    left_x = left_wall+1 if left_wall else left_spill
                    right_x = right_wall if right_wall else right_spill+1
                    for spill_x in range(left_x, right_x):
                        grid[y][spill_x] = '|'
                    if left_spill is not None:
                        recursive_fill(left_spill, y)
                    if right_spill is not None:
                        recursive_fill(right_spill, y)
                    y -= 1
            # if nothing go back up
            else:
                y -= 1
    recursive_fill(x_src, y_src)
    return (grid, offset)

In [163]:
def count_total_filled(grid, tiles='+|~'):
    total_filled = 0
    for row in grid:
        for char in row:
            if char in tiles:
                total_filled += 1
    return total_filled

In [156]:
filepath = 'data/day17.txt'
#filepath = 'data/day17_mini.txt'
grid, (x_off, y_off) = read_grid_from_file(filepath)
# raw_filepath = 'data/day17_test_grid.txt'
# grid, (x_off, y_off) = read_raw_grid_from_file(raw_filepath)
width, height = len(grid[0]), len(grid)

In [157]:
write_grid_to_file('data/day17_grid.txt', grid)

In [158]:
filled_grid, (x_off, y_off) = fill_grid(grid, (x_off, y_off), (500, 0))

In [159]:
write_grid_to_file('data/day17_filled_grid.txt', filled_grid)

In [165]:
total_tiles_filled = count_total_filled(filled_grid)
print('Total filled: {}'.format(total_tiles_filled))

Total filled: 32552


In [166]:
total_static = count_total_filled(filled_grid, tiles='~')
print('Total static: {}'.format(total_static))

Total static: 26405
